In [1]:
# import os
# import numpy as np
import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch_geometric.nn import ChebConv  # Using Chebyshev convolution
# from torch_geometric.nn import knn_graph
# from torch_geometric.utils import get_laplacian, get_mesh_laplacian
# from torch_sparse import coalesce
# from torch_geometric.utils import degree

from points_dataset import EmbedderDataset
from torch_geometric.data import Data, DataLoader

from models import *
from point_cloud_datasest import PointCloudDataset
from torch.utils.tensorboard import SummaryWriter
torch.__version__, "cuda available" if torch.cuda.is_available() else "cpu only"

('2.4.0+cu124', 'cuda available')

In [2]:
device = "cpu" if not torch.cuda.is_available() else "cuda"

# ds_path = r"C:\Projects\multiSetup\datasets_embedded\ds_1024.pth"
point_cloud_ds_path = r"C:\Projects\multiSetup\datasets_embedded\ds_256.pth"
point_cloud_ds = torch.load(point_cloud_ds_path)
point_cloud_dataset_data = point_cloud_ds.data

print(f"Using {device} device, ds length - {len(point_cloud_ds)}, data.shape - {point_cloud_dataset_data.shape}")

EPOCHS = 200
BATCH_SIZE = 128
SPLIT_FACTOR = .8
TRAIN_MODE = (False, True)[1]
REMOVE_OLD_MODELS = True
POINTS_PER_SHAPE = 256 # 1024
POINT_DIM = 3


C:\Users\5010858\AppData\Local\Temp/ipykernel_21880/1557303662.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  point_cloud_ds = torch.load(point_cloud_ds_path)


Using cuda device, ds length - 27571, data.shape - (27571, 768)


In [3]:
diffision_ae_ds = PointCloudDataset(torch.from_numpy(point_cloud_dataset_data), k=6)
len(diffision_ae_ds)
train_loader = DataLoader(diffision_ae_ds, batch_size=BATCH_SIZE, shuffle=True)

c:\Python39\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [4]:
# batch = next(iter(train_loader))
# batch # DataBatch(x=[131072, 3], edge_index=[2, 786432], edge_weight=[786432], batch=[131072], ptr=[129])

In [5]:
def train_autoencoder(model, train_loader, device, epochs=EPOCHS, lr=1e-3, log_dir="runs/autoencoder"):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir=log_dir)

    best_loss = float('inf')  # Initialize best loss to infinity
    best_model_path = None

    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch_idx, batch in enumerate(train_loader):
            x = batch.x  # Node features for all graphs in the batch
            edge_index = batch.edge_index  # Edge indices for all graphs in the batch
            edge_weight = batch.edge_weight  # Edge weights (Laplacian) for all graphs in the batch
            model.train()
            optimizer.zero_grad()

            # Move data to the specified device (GPU or CPU)
            x = x.to(device)  # torch.Size([128, 1024, 3])
            edge_index = edge_index.to(device)  # torch.Size([128, 2, 6144])
            edge_weight = edge_weight.to(device)

            # Forward pass
            reconstructed, _ = model(x, edge_index, edge_weight=edge_weight)
            loss = loss_fn(reconstructed, x)

            # Backward pass
            loss.backward()
            optimizer.step()

            # Accumulate loss for the epoch
            epoch_loss += loss.item()

            # Log batch-level loss to TensorBoard
            writer.add_scalar('Loss/train_batch', loss.item(), epoch * len(train_loader) + batch_idx)

        # Average loss per epoch
        avg_epoch_loss = epoch_loss / len(train_loader)

        # Log epoch-level loss to TensorBoard
        writer.add_scalar('Loss/train_epoch', avg_epoch_loss, epoch)

        # Print progress
        print(f'Epoch {epoch}/{epochs}, Loss: {avg_epoch_loss}')

        # Check if the current model is the best one (based on loss)
        if avg_epoch_loss < best_loss:
            best_loss = avg_epoch_loss
            best_model_path = f"models/best_model_epoch_{epoch}.pth"
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved with loss {best_loss} at epoch {epoch}")

    # Close TensorBoard writer when training is complete
    writer.close()

    # Return the path to the best model for further usage
    return best_model_path

# Initialize and train the model
hidden_features = 64
latent_dim = 32

model = DiffusionNetAutoencoder(POINT_DIM, hidden_features, latent_dim).to(device)

# Call the training function with TensorBoard logging
best_model_path = train_autoencoder(model, train_loader, device)
print(f"Best model saved at: {best_model_path}")

Epoch 0/200, Loss: 28.781465234579862
Best model saved with loss 28.781465234579862 at epoch 0
Epoch 1/200, Loss: 3.074230440236904
Best model saved with loss 3.074230440236904 at epoch 1
Epoch 2/200, Loss: 1.762163571185536
Best model saved with loss 1.762163571185536 at epoch 2
Epoch 3/200, Loss: 1.1879559239303623
Best model saved with loss 1.1879559239303623 at epoch 3
Epoch 4/200, Loss: 0.8996810951718578
Best model saved with loss 0.8996810951718578 at epoch 4
Epoch 5/200, Loss: 0.7269688036154818
Best model saved with loss 0.7269688036154818 at epoch 5
Epoch 6/200, Loss: 0.6285310008183673
Best model saved with loss 0.6285310008183673 at epoch 6
Epoch 7/200, Loss: 0.5189799978225319
Best model saved with loss 0.5189799978225319 at epoch 7
Epoch 8/200, Loss: 0.45530141338154123
Best model saved with loss 0.45530141338154123 at epoch 8
Epoch 9/200, Loss: 0.41178892870192174
Best model saved with loss 0.41178892870192174 at epoch 9
Epoch 10/200, Loss: 0.360662590160414
Best model s